<a href="https://colab.research.google.com/github/moosunny/Transformer_Translator/blob/main/Seq2Seq_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from transformers import MarianMTModel, MarianTokenizer, AutoTokenizer # MT: Machine Translation
import pandas as pd
from tqdm import tqdm
import math, random

# 한국어 토크나이저 활용(upstage)
tokenizer = AutoTokenizer.from_pretrained("upstage/solar-1-mini-tokenizer")
# 패딩을 오른쪽으로 설정
tokenizer.padding_side = "right"


# 한국어-영어 대화체 데이터 다운로드(link:)
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Transformer_Translation/2_대화체.xlsx')
data

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,대분류,소분류,상황,Set Nr.,발화자,원문,번역문
0,비즈니스,회의,의견 교환하기,1,A-1,이번 신제품 출시에 대한 시장의 반응은 어떤가요?,How is the market's reaction to the newly rele...
1,비즈니스,회의,의견 교환하기,1,B-1,판매량이 지난번 제품보다 빠르게 늘고 있습니다.,The sales increase is faster than the previous...
2,비즈니스,회의,의견 교환하기,1,A-2,그렇다면 공장에 연락해서 주문량을 더 늘려야겠네요.,"Then, we'll have to call the manufacturer and ..."
3,비즈니스,회의,의견 교환하기,1,B-2,"네, 제가 연락해서 주문량을 2배로 늘리겠습니다.","Sure, I'll make a call and double the volume o..."
4,비즈니스,회의,의견 교환하기,2,A-1,지난 회의 마지막에 논의했던 안건을 다시 볼까요?,Shall we take a look at the issues we discusse...
...,...,...,...,...,...,...,...
99995,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",24999,B-2,"저희가 가격표 배치를 잘못해서 혼동을 드렸나 봐요, 죄송해요.",It seems that we didn't place the price tags c...
99996,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",25000,A-1,"백화점 포인트로 계산하고 싶은데, 가능한가요?",Can I pay using the department store points?
99997,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",25000,B-1,"네, 물론이죠, 전화번호 입력해주시면 됩니다.","Yes, of course, you just need to enter your ph..."
99998,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",25000,A-2,"입력했어요, 전액 백화점 포인트로 결제하고 싶어요.","I entered it, I want to pay it with all the de..."


In [3]:
eos_idx = tokenizer.eos_token_ids
pad_idx = tokenizer.pad_token_ids
unk_idx = tokenizer.unk_token_ids


# eos token과 pad 토큰의 인덱스가 동일함을 확인 -> unk 토큰을 활용한 문장 패딩 수행
print(eos_idx)
print(pad_idx)
print(unk_idx)

2
2
0


In [4]:
tokenizer.pad_token = tokenizer.unk_token
print(tokenizer.pad_token)

pad_idx = tokenizer.pad_token_id
print(pad_idx)

<unk>
0


In [5]:
# 하이퍼 파라미터 모음
max_len = 100
batch_size = 128
vocab_size = tokenizer.vocab_size # 64000개

EPOCH = 15
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
embedding_dim = 512

In [6]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return self.data.loc[idx, "원문"], self.data.loc[idx, "번역문"]

custom_DS = CustomDataset(data)
print(custom_DS[0])

train_DS, val_DS, test_DS = torch.utils.data.random_split(custom_DS, [97000, 2000, 1000])
# 논문에서는 450만개 영,독 문장 pair 사용

train_DL = torch.utils.data.DataLoader(train_DS, batch_size=batch_size, shuffle=True)
val_DL = torch.utils.data.DataLoader(val_DS, batch_size=batch_size, shuffle=True)
test_DL = torch.utils.data.DataLoader(test_DS, batch_size=batch_size, shuffle=True)

print(len(train_DS))
print(len(val_DS))
print(len(test_DS))

('이번 신제품 출시에 대한 시장의 반응은 어떤가요?', "How is the market's reaction to the newly released product?")
97000
2000
1000


In [7]:
i = 5
src_text, trg_text  = test_DS[0]
print(f"인덱스:{test_DS.indices[i]}")
print(f"원문:{src_text}")
print(f"원문:{trg_text}")

인덱스:94670
원문:그렇게 생각하시면 손님들이 더 올 수밖에 없네요.
원문:If you think so, there's no choice but to have more guests.


In [8]:
eos_idx = tokenizer.eos_token_ids
print(eos_idx)

for src_text, trg_text in train_DL:
  print(src_text)
  print(trg_text)
  print(len(src_text))
  print(len(trg_text))

  src = tokenizer(src_text, padding = True, truncation=True, max_length=max_len, return_tensors = 'pt', add_special_tokens = False).input_ids
  trg_text = [s + tokenizer.eos_token for s in trg_text] # eos token 추가 필요
  trg = tokenizer(trg_text, padding = True, truncation=True, max_length=max_len, return_tensors = 'pt', add_special_tokens = True).input_ids

  print(src[:2])
  print(trg[:2])
  print(src.shape)
  print(trg.shape)

  print(trg[:, -1])
  print(tokenizer.decode(trg[trg[:,-1]==pad_idx,:][0]))
  print(trg[5, :-1])
  print(trg[5, 1:])

  break

2
('50만 원 환전하고 싶은데, 도와주세요.', '혹시 재입고 될 예정인가요? 꼭 사고 싶은데.', '저번주 판매량이 갑자기 급증한 원인이 뭔가요?', '본인이 직접 사용하시는 건가요?', '분실에 책임을 안 진다는 건 읽었는데, 그래도 좀 도와주세요.', '일단 성적을 제대로 낸 건지 확인해보고 메일로 연락해줄게.', '항공사마다 규정이 다르므로 제가 답해드릴 수는 없습니다.', '그럼 각자 읽어 보시고 다음 회의 때 얘기합시다.', '건너서 좌회전하면 버스 정류장 하나가 있기는 해요.', '하루에 1시간 정도 운동하는 데 시간을 할애했습니다.', '이번에 경쟁사가 아주 제대로 준비한 모양이야.', '사실 여기에 오는 게 많은 어린이와 어른들의 꿈이지.', '마루의 틈이 벌어지거나 엇갈리진 않습니까?', '그 배우가 이번에 감독이랑 싸워서 중간에 하차한대.', '접수 처리되었으니 번호를 호명할 때까지 기다려주세요.', '하지만 짜게 먹는 건 몸에 좋지 않습니다.', '하지만 우리 빨리 버스를 타야 제시간에 도착하잖아.', '콜라 하나랑 사이다 하나 주세요, 아 혹시 리필되나요?', '처음에 계약서 보낼 때 수량을 잘못 써서 보냈죠?', '그러자, 영화 시간 많이 안 남았어, 얼른 가자.', '구매자와 회의 일정을 잡으려 하고 있어서 그런데 언제가 편한지 알려주세요.', '혹시 환승해야 성당으로 갈 수 있나요?', '세세한 부분은 차치하고서라도 납품 기한 부분 같은 것도 잘못되어 있어서요.', '오늘이나 내일 중에 본사에 가시는 분이 있나요?', '그럼요, 그럼 식사랑 같이 준비해드릴게요.', '잠깐이면 되니까, 이걸 푸는 방법만 알려주고 가.', '그럼 지금 제가 빨리 가서 음식을 가져오겠습니다.', '네가 성적을 잘 받는 비결이 뭐니, 나도 배우고 싶어.', '하지만 운동을 정말 싫어하는데, 어떻게 하면 좋을까요?', '계산은 후불로도 가능한데 후불로 도와드리면 될까요?', '저기요, 이쪽에 남은 음식은 포장해주시고 계산도 해주세요.', '어떤 음식을 

## 모델 학습 예시

In [9]:
embedding_dim = 1000 # Seq2Seq 논문 상으로 1000 Embedding_dim
teacher_forcing_ratio = 0.5

### Encoder
enc_embedding = nn.Embedding(vocab_size, embedding_dim)
enc_dropout = nn.Dropout(0.5)
enc_lstm = nn.LSTM(input_size = embedding_dim, hidden_size = 1000, num_layers = 4, dropout = 0.5, batch_first= True)

### Decoder
dec_embedding = nn.Embedding(vocab_size, embedding_dim)
dec_dropout = nn.Dropout(0.5)
dec_lstm = nn.LSTM(input_size = embedding_dim, hidden_size = 1000, num_layers = 4, dropout = 0.5)
dec_fc = nn.Linear(1000, vocab_size)



for src, trg in train_DL:
  # print(src[0]) # (배치 사이즈, 문장 길이)
  # print(trg[0])
  # N = len(train_DL)
  src = tokenizer(src, padding = True, truncation = True, max_length = max_len, return_tensors = 'pt', add_special_tokens = False).input_ids
  trg = [s + tokenizer.eos_token for s in trg] # eos token 추가 필요
  trg = tokenizer(trg, padding = True, truncation = True, max_length = max_len, return_tensors = 'pt', add_special_tokens = True).input_ids
  # print(src.shape)
  # print(trg.shape)

  ### Encoder Layer
  enc_embedded_src = enc_embedding(src)
  # print(f"임베딩 후 차원: {enc_embedded_src.shape}") # (배치 사이즈, 문장 길이, 임베딩 차원)
  enc_src = enc_dropout(enc_embedded_src)
  # print(f"임베딩 -> Dropout 후 차원: {enc_src.shape}")

  # encoder output
  enc_output, (enc_hidden, enc_cell) = enc_lstm(enc_src)
  # print(f"LSTM 레이어 통과 후 차원: {enc_output.shape}")
  #  # num_layers 만큼 hidden, cell state 출력
  # print(f"hidden state 차원: {enc_hidden.shape}")
  # print(f"cell state 차원: {enc_cell.shape}")

  ### Decoder Layer
  # input = [batch size]
  # hidden = [n layers * n directions, batch size, hidden dim]
  # cell = [n layers * n directions, batch size, hidden dim]
  # n directions in the decoder will both always be 1, therefore:
  # hidden = [n layers, batch size, hidden dim]
  # context = [n layers, batch size, hidden dim]
  trg = trg.permute(1,0)
  print(trg.shape)
  trg_length = trg.shape[0]
  outputs = torch.zeros(trg_length, batch_size, vocab_size)
  dec_input =  trg[0, :] # sos token
  dec_input = dec_input.unsqueeze(0)
  for t in range(1, trg_length):
    dec_embedded = dec_dropout(dec_embedding(dec_input))
    dec_output, (dec_hidden, dec_cell) = dec_lstm(dec_embedded, (enc_hidden, enc_cell))
    prediction = dec_fc(dec_output.squeeze(0))
    outputs[t] = prediction
    # decide if we are going to use teacher forcing or not
    teacher_force = random.random() < teacher_forcing_ratio
    # get the highest predicted token from our predictions
    top1 = prediction.argmax(1)
    # if teacher forcing, use actual next token as next input
    # if not, use predicted token
    # teacher forcing 여부에 따라 다음 입력 결정하고, 반드시 unsqueeze(0)로 차원을 맞춤.
    dec_input = (trg[t, :] if teacher_force else top1).unsqueeze(0)
    # print(dec_input.shape)
    # input = [batch size]
  print(outputs.shape)
  output_dim = outputs.shape[-1]
  outputs = outputs[1:].view(-1, output_dim)
  print(outputs.shape)
  # output = [(trg length - 1) * batch size, trg vocab size]
  print(trg.shape)
  trg = trg[1:].contiguous().view(-1)
  # trg = [(trg length - 1) * batch size]
  loss = criterion(outputs, trg)
  print(loss)

  break

torch.Size([32, 128])
torch.Size([32, 128, 64000])
torch.Size([3968, 64000])
torch.Size([32, 128])
tensor(11.0704, grad_fn=<NllLossBackward0>)


# Seq2Seq 구현

In [10]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first= True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [batch size, seq_len]
        embedded = self.dropout(self.embedding(src))
        # embedded = [batch size, seq_len, embedding dim]
        outputs, (hidden, cell) = self.lstm(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hidden dim]
        # context = [n layers, batch size, hidden dim]
        # input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        # output = [seq length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [batch size, seq_len]
        # trg = [batch size, seq_len]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        trg = trg.permute(1,0)
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        dec_input = trg[0, :]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)

        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        dec_input = dec_input.unsqueeze(0)
        for t in range(1, trg_length):
          dec_output, hidden, cell = self.decoder(dec_input, hidden, cell)
          outputs[t] = dec_output
          # decide if we are going to use teacher forcing or not
          teacher_force = random.random() < teacher_forcing_ratio
          # get the highest predicted token from our predictions
          top1 = dec_output.argmax(1)
          # if teacher forcing, use actual next token as next input
          # if not, use predicted token
          # teacher forcing 여부에 따라 다음 입력 결정하고, 반드시 unsqueeze(0)로 차원을 맞춤.
          dec_input = trg[t].unsqueeze(0) if teacher_force else top1.unsqueeze(0)

          # dec_input = (trg[t, :] if teacher_force else top1).unsqueeze(0)


        return outputs

In [11]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

max_len = 50

# seed 고정
def seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed(42)

input_dim = tokenizer.vocab_size
output_dim = tokenizer.vocab_size
encoder_embedding_dim = 256
decoder_embedding_dim = 256

In [12]:
tokenizer.vocab_size

64000

In [13]:
hidden_dim = 512
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)

model = Seq2Seq(encoder, decoder, device).to(device)

cuda


In [14]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)


model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(64000, 1000)
    (lstm): LSTM(1000, 1000, num_layers=4, batch_first=True, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(64000, 1000)
    (lstm): LSTM(1000, 1000, num_layers=4, dropout=0.5)
    (fc_out): Linear(in_features=1000, out_features=64000, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [15]:
print(input_dim,"input_dim")
print(output_dim,"output_dim")

64000 input_dim
64000 output_dim


In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 256,128,000 trainable parameters


In [17]:
pad_index = tokenizer.pad_token_id
print(pad_index)

0


In [18]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

def train_fn(model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device):
    model.train()
    epoch_loss = 0
    for src, trg in tqdm(data_loader):
        src = tokenizer(src, padding = True, truncation = True, max_length = max_len, return_tensors = 'pt', add_special_tokens = False).input_ids
        trg = [s + tokenizer.eos_token for s in trg] # eos token 추가 필요
        trg = tokenizer(trg, padding = True, truncation = True, max_length = max_len, return_tensors = 'pt', add_special_tokens = True).input_ids
        src, trg = src.to(device), trg.to(device)
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)

        # output = [(trg length - 1) * batch size, trg vocab size]

        trg = trg.permute(1, 0)
        trg = trg[1:].contiguous().view(-1)
        # trg = [(trg length - 1) * batch size]

        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [19]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for src, trg in data_loader:
            src = tokenizer(src, padding = True, truncation = True, max_length = max_len, return_tensors = 'pt', add_special_tokens = False).input_ids
            trg = [s + tokenizer.eos_token for s in trg] # eos token 추가 필요
            trg = tokenizer(trg, padding = True, truncation = True, max_length = max_len, return_tensors = 'pt', add_special_tokens = True).input_ids
            src, trg = src.to(device), trg.to(device)

            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)

            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg.permute(1, 0)
            trg = trg[1:].contiguous().view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [20]:
from tqdm.notebook import tqdm

n_epochs = 10
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_DL,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        val_DL,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "tut1-model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/758 [00:00<?, ?it/s]

torch.Size([128, 27])
torch.Size([128, 37])
torch.Size([4608, 64000])
torch.Size([128, 26])
torch.Size([128, 50])
torch.Size([6272, 64000])
torch.Size([128, 30])
torch.Size([128, 34])
torch.Size([4224, 64000])
torch.Size([128, 27])
torch.Size([128, 31])
torch.Size([3840, 64000])
torch.Size([128, 31])
torch.Size([128, 42])
torch.Size([5248, 64000])
torch.Size([128, 38])
torch.Size([128, 35])
torch.Size([4352, 64000])
torch.Size([128, 25])
torch.Size([128, 35])
torch.Size([4352, 64000])
torch.Size([128, 32])
torch.Size([128, 32])
torch.Size([3968, 64000])
torch.Size([128, 33])
torch.Size([128, 33])
torch.Size([4096, 64000])
torch.Size([128, 34])
torch.Size([128, 38])
torch.Size([4736, 64000])
torch.Size([128, 25])
torch.Size([128, 31])
torch.Size([3840, 64000])
torch.Size([128, 43])
torch.Size([128, 42])
torch.Size([5248, 64000])
torch.Size([128, 24])
torch.Size([128, 36])
torch.Size([4480, 64000])
torch.Size([128, 33])
torch.Size([128, 35])
torch.Size([4352, 64000])
torch.Size([128, 30]

  0%|          | 0/758 [00:00<?, ?it/s]

torch.Size([128, 38])
torch.Size([128, 40])
torch.Size([4992, 64000])
torch.Size([128, 27])
torch.Size([128, 36])
torch.Size([4480, 64000])
torch.Size([128, 43])
torch.Size([128, 42])
torch.Size([5248, 64000])
torch.Size([128, 33])
torch.Size([128, 42])
torch.Size([5248, 64000])
torch.Size([128, 31])
torch.Size([128, 37])
torch.Size([4608, 64000])
torch.Size([128, 27])
torch.Size([128, 35])
torch.Size([4352, 64000])
torch.Size([128, 25])
torch.Size([128, 32])
torch.Size([3968, 64000])
torch.Size([128, 25])
torch.Size([128, 37])
torch.Size([4608, 64000])
torch.Size([128, 27])
torch.Size([128, 34])
torch.Size([4224, 64000])
torch.Size([128, 33])
torch.Size([128, 36])
torch.Size([4480, 64000])
torch.Size([128, 34])
torch.Size([128, 50])
torch.Size([6272, 64000])
torch.Size([128, 24])
torch.Size([128, 34])
torch.Size([4224, 64000])
torch.Size([128, 30])
torch.Size([128, 38])
torch.Size([4736, 64000])
torch.Size([128, 28])
torch.Size([128, 33])
torch.Size([4096, 64000])
torch.Size([128, 23]

...

torch.Size([128, 38])
torch.Size([128, 36])
torch.Size([4480, 64000])
torch.Size([128, 25])
torch.Size([128, 31])
torch.Size([3840, 64000])
torch.Size([128, 27])
torch.Size([128, 34])
torch.Size([4224, 64000])
torch.Size([128, 33])
torch.Size([128, 40])
torch.Size([4992, 64000])
torch.Size([128, 26])
torch.Size([128, 33])
torch.Size([4096, 64000])
torch.Size([128, 43])
torch.Size([128, 42])
torch.Size([5248, 64000])
torch.Size([128, 33])
torch.Size([128, 33])
torch.Size([4096, 64000])
torch.Size([128, 32])
torch.Size([128, 38])
torch.Size([4736, 64000])
torch.Size([128, 34])
torch.Size([128, 38])
torch.Size([4736, 64000])
torch.Size([128, 31])
torch.Size([128, 31])
torch.Size([3840, 64000])
torch.Size([128, 27])
torch.Size([128, 50])
torch.Size([6272, 64000])
torch.Size([128, 25])
torch.Size([128, 31])
torch.Size([3840, 64000])
torch.Size([128, 31])
torch.Size([128, 42])
torch.Size([5248, 64000])
torch.Size([128, 27])
torch.Size([128, 34])
torch.Size([4224, 64000])
torch.Size([128, 31]

  0%|          | 0/758 [00:00<?, ?it/s]

torch.Size([128, 34])
torch.Size([128, 38])
torch.Size([4736, 64000])
torch.Size([128, 26])
torch.Size([128, 31])
torch.Size([3840, 64000])
torch.Size([128, 26])
torch.Size([128, 33])
torch.Size([4096, 64000])
torch.Size([128, 33])
torch.Size([128, 35])
torch.Size([4352, 64000])
torch.Size([128, 27])
torch.Size([128, 37])
torch.Size([4608, 64000])
torch.Size([128, 43])
torch.Size([128, 42])
torch.Size([5248, 64000])
torch.Size([128, 27])
torch.Size([128, 35])
torch.Size([4352, 64000])
torch.Size([128, 23])
torch.Size([128, 34])
torch.Size([4224, 64000])
torch.Size([128, 30])
torch.Size([128, 37])
torch.Size([4608, 64000])
torch.Size([128, 28])
torch.Size([128, 50])
torch.Size([6272, 64000])
torch.Size([128, 32])
torch.Size([128, 33])
torch.Size([4096, 64000])
torch.Size([128, 38])
torch.Size([128, 36])
torch.Size([4480, 64000])
torch.Size([128, 29])
torch.Size([128, 40])
torch.Size([4992, 64000])
torch.Size([128, 24])
torch.Size([128, 30])
torch.Size([3712, 64000])
torch.Size([128, 33]

  0%|          | 0/758 [00:00<?, ?it/s]

torch.Size([128, 31])
torch.Size([128, 42])
torch.Size([5248, 64000])
torch.Size([128, 29])
torch.Size([128, 30])
torch.Size([3712, 64000])
torch.Size([128, 43])
torch.Size([128, 42])
torch.Size([5248, 64000])
torch.Size([128, 27])
torch.Size([128, 36])
torch.Size([4480, 64000])
torch.Size([128, 30])
torch.Size([128, 38])
torch.Size([4736, 64000])
torch.Size([128, 27])
torch.Size([128, 37])
torch.Size([4608, 64000])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([4736, 64000])
torch.Size([128, 26])
torch.Size([128, 30])
torch.Size([3712, 64000])
torch.Size([128, 27])
torch.Size([128, 35])
torch.Size([4352, 64000])
torch.Size([128, 33])
torch.Size([128, 34])
torch.Size([4224, 64000])
torch.Size([128, 33])
torch.Size([128, 34])
torch.Size([4224, 64000])
torch.Size([128, 25])
torch.Size([128, 36])
torch.Size([4480, 64000])
torch.Size([128, 26])
torch.Size([128, 34])
torch.Size([4224, 64000])
torch.Size([128, 29])
torch.Size([128, 35])
torch.Size([4352, 64000])
torch.Size([128, 29]

In [22]:
model.load_state_dict(torch.load("tut1-model.pt"))

test_loss = evaluate_fn(model, test_DL, criterion, device)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

<ipython-input-22-621d125867ee>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("tut1-model.pt"))


torch.Size([128, 36])
torch.Size([128, 35])
torch.Size([4352, 64000])
torch.Size([128, 38])
torch.Size([128, 40])
torch.Size([4992, 64000])
torch.Size([128, 25])
torch.Size([128, 32])
torch.Size([3968, 64000])
torch.Size([128, 27])
torch.Size([128, 33])
torch.Size([4096, 64000])
torch.Size([128, 28])
torch.Size([128, 38])
torch.Size([4736, 64000])
torch.Size([128, 28])
torch.Size([128, 35])
torch.Size([4352, 64000])
torch.Size([128, 26])
torch.Size([128, 37])
torch.Size([4608, 64000])
torch.Size([104, 25])
torch.Size([104, 30])
torch.Size([3016, 64000])
| Test Loss: 5.016 | Test PPL: 150.766 |


In [75]:
def translate_sentence(
    sentence,
    model,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
      tokens = tokenizer(sentence, padding = True, truncation = True, max_length = max_len, return_tensors = 'pt', add_special_tokens = False).input_ids
      print(tokens.shape)
      tensor = torch.LongTensor(tokens).to(device)
      hidden, cell = model.encoder(tensor)
      inputs = [tokenizer.bos_token_ids]
      print(inputs[-1])
      for _ in range(max_output_length):
          inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
          output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
          predicted_token = output.argmax(-1).item()
          inputs.append(predicted_token)
          if predicted_token == eos_token:
              break
      tokens = tokenizer.decode(inputs)
    return tokens

In [89]:
def translate_fn(model, tokenizer, sentence, device, max_len=100):
    model.eval()

    # 🔹 1. 입력 문장 토큰화
    src = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True, max_length=max_len, add_special_tokens=False).input_ids.to(device)
    # src = src.to(device)

    # 🔹 2. 인코더에 입력하여 초기 hidden state 얻기
    with torch.no_grad():
        hidden, cell = model.encoder(src)

    # 🔹 3. <sos> 토큰으로 디코딩 시작
    sos_token = tokenizer.bos_token  # 모델에 따라 다름
    eos_token = tokenizer.eos_token_id

    dec_input = torch.tensor([[sos_token]], device=device)  # [1, 1] 크기의 텐서
    output_tokens = []

    # 🔹 4. Autoregressive Decoding (Greedy Decoding)
    for _ in range(max_len):
        with torch.no_grad():
            dec_output, hidden, cell = model.decoder(dec_input, hidden, cell)

        pred_token = dec_output.argmax(2)  # [1, 1] 크기의 텐서
        pred_token_id = pred_token.item()
        output_tokens.append(pred_token_id)

        # 🔹 5. <eos> 토큰이 나오면 종료
        if pred_token_id == eos_token:
            break

        # 🔹 6. 다음 입력으로 사용
        dec_input = pred_token

    # 🔹 7. 토큰을 텍스트로 변환
    translated_sentence = tokenizer.decode(output_tokens, skip_special_tokens=True)

    return translated_sentence

In [90]:
sentence = "Hello, how are you?"  # 번역할 문장
translated_sentence = translate_fn(model, tokenizer, sentence, device)
print(f"Translated: {translated_sentence}")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [76]:
sentence = test_DS[0][0]
expected_translation = test_DS[0][1]

sentence, expected_translation

('그렇게 생각하시면 손님들이 더 올 수밖에 없네요.',
 "If you think so, there's no choice but to have more guests.")

In [77]:
translation = translate_sentence(
    sentence,
    model,
    tokenizer.bos_token,
    tokenizer.eos_token,
    device,
)

torch.Size([1, 11])


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
sentence = test_data[0]["de"]
expected_translation = test_data[0]["en"]

sentence, expected_translation